# Introduction to Bedrock - Generating images using Stable Diffusion

--- 

In this demo notebook, we demonstrate how to use the Bedrock Python SDK for a image generation task. We show how to use The Stable Diffusion foundational model to create images

---

Note: This notebook was tested in Amazon SageMaker Studio with Python 3 (Data Science 2.0) kernel.

1. [Set Up](#1.-Set-Up-and-API-walkthrough)
2. [Image Creation](#2.-Generate-Images-with-Text)
3. [Image to Image](#3.-Image-to-Image) 

## 1. Set Up and API walkthrough

---
Before executing the notebook for the first time, execute this cell to add bedrock extensions to the Python boto3 SDK

---

**Select region: "us-east-1"(M1), "us-west-2"(M2)**

In [2]:
bedrock_region = "us-west-2" 

In [3]:
if bedrock_region == "us-east-1":    
    bedrock_config = {
        "region_name":bedrock_region,
        "endpoint_url":"https://bedrock.us-east-1.amazonaws.com"
    }
elif bedrock_region == "us-west-2":  
    bedrock_config = {
        "region_name":bedrock_region,
        "endpoint_url":"https://prod.us-west-2.frontend.bedrock.aws.dev"
    }

#### Now let's set up our connection to the Amazon Bedrock SDK using Boto3

In [4]:
import boto3
import json
bedrock = boto3.client(
    service_name='bedrock',
    region_name=bedrock_config["region_name"],
    endpoint_url=bedrock_config["endpoint_url"]
)

#### We can validate our connection by testing out the _list_foundation_models()_ method, which will tell us all the models available for us to use 

In [5]:
bedrock.list_foundation_models()

{'ResponseMetadata': {'RequestId': '2ab41763-0673-47b7-b21f-bc57a0f96658',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 14 Jul 2023 04:49:06 GMT',
   'content-type': 'application/json',
   'content-length': '861',
   'connection': 'keep-alive',
   'x-amzn-requestid': '2ab41763-0673-47b7-b21f-bc57a0f96658'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-tg1-large',
   'modelId': 'amazon.titan-tg1-large'},
  {'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-e1t-medium',
   'modelId': 'amazon.titan-e1t-medium'},
  {'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/stability.stable-diffusion-xl',
   'modelId': 'stability.stable-diffusion-xl'},
  {'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/ai21.j2-grande-instruct',
   'modelId': 'ai21.j2-grande-instruct'},
  {'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/ai21.j2-jumbo-instruct',
   'modelId': 'ai21.j2-jumbo-in

#### In this Notebook we will be using the invoke_model() method of Amazon Bedrock. This will be the primary method we use for most of our Image  Generation and Processing tasks. 

##### The mandatory parameters required to use this method are, where _modelId_ represents the Amazon Bedrock model ARN, and _body_ which is the prompt for our task. The _body_ prompt will change depending on the foundational model provider selected. We walk through this in detail below

```
{
   modelId= model_id,
   contentType= "application/json",
   accept= "application/json",
   body=body
}

```

## 2. Generate Images with Text

#### Let's now try out the Stable Diffusion model in Amazon Bedrock to generate some images with text

In [6]:
prompt_data ="""Middle age man walking through time square on a snowy day""" #If you'd like to try your own prompt, edit this parameter!

In [7]:
from botocore.config import Config
from botocore.exceptions import ClientError
import json
from PIL import Image
from io import BytesIO
import base64
from base64 import b64encode
from base64 import b64decode
import boto3


body = json.dumps({"text_prompts":[{"text":prompt_data}]}) 
modelId = 'stability.stable-diffusion-xl'
accept = 'application/json'
contentType = 'application/json'

response = bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response = json.loads(response.get('body').read())
images = response.get('artifacts')

image = Image.open(BytesIO(b64decode(images[0].get('base64'))))
image.save("generated_image.png")

AccessDeniedException: An error occurred (AccessDeniedException) when calling the InvokeModel operation: Your account is not authorized to invoke this API operation.

## 3. Image to Image

#### Stable Diffusion let's us do some interesting stuff with our images like adding new characters or modifying scenery let's give it a try

In [5]:
prompt_data = """Change the character to a grandma and her grandaughter on a snowy day""" #If you'd like to try your own prompt, edit this parameter!

In [6]:
buffer = BytesIO()
img = Image.open("generated_image.png")
img.save(buffer, format="PNG")
img_bytes = buffer.getvalue()

body = json.dumps({"text_prompts":[{"text": prompt_data }], "init_image": base64.b64encode(img_bytes).decode()})
modelId = 'stability.stable-diffusion-xl'

try: 
    response = bedrock.invoke_model(body=body, modelId=modelId, contentType="application/json", accept="image/png")
except ClientError as error:
    print(error.response)
 
if response['contentType'] == 'image/png':
    # Get the response body as bytes
    image_data = response['body'].read()
else:
    image_data = response['body']

image = Image.open(BytesIO(image_data))
file_name = 'converted.png'
#save file
image.save(file_name)